In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd


In [43]:
CEs = pd.read_csv('CEs_PCSE.csv', sep=';', index_col=0, decimal=',')
CEs['meanCE'] = CEs.loc[:,['CE_rcp85','CE_rcp45','CE_rcp26']].mean(axis=1)
CEs['varCE'] = CEs.loc[:,['CE_rcp85','CE_rcp45','CE_rcp26']].var(axis=1)
CEs.head()

,wheat_perc,maize_perc,hay_perc,CE_rcp85,CE_rcp45,CE_rcp26,meanCE,varCE
hay.0_wheat.20_maize.80,20,80,0,31.115724,34.913673,36.495480,34.174959,7.644719
hay.0_wheat.10_maize.90,10,90,0,30.861261,34.853083,36.565591,34.093311,8.567786
hay.0_wheat.0_maize.100,0,100,0,30.620022,34.796495,36.655081,34.023866,9.553200
hay.10_wheat.20_maize.70,20,70,10,30.689424,34.409415,35.951629,33.683489,7.317924
hay.10_wheat.10_maize.80,10,80,10,30.413487,34.329397,35.976871,33.573252,8.166628


In [56]:
from plotly.colors import n_colors, hex_to_rgb

#1b9e77
#d95f02
#7570b3
color1 = hex_to_rgb('#1b9e77')
color2 = hex_to_rgb('#d95f02')
color3 = hex_to_rgb('#7570b3')
colors = [color1, color2, color3]

def get_colors(n, breakpoints=['#1b9e77', '#d95f02', '#7570b3']):
    half_1 = int(n / 2)
    half_2 = n - half_1

    col_1 = n_colors(hex_to_rgb(breakpoints[0]), hex_to_rgb(breakpoints[1]), half_1, 'hex')
    col_2 = n_colors(hex_to_rgb(breakpoints[1]), hex_to_rgb(breakpoints[2]), half_2, 'hex')
    
    return [*col_1, *col_2]


def mix_color(weights, as_tuple=False, colors=['#1b9e77', '#d95f02', '#7570b3']):
    """mix the colors given the weights"""
    colors = [hex_to_rgb(c) for c in colors]
    r = sum([(w / 100) * c[0] for w, c in zip(weights, colors)])
    g = sum([(w / 100) * c[1] for w, c in zip(weights, colors)])
    b = sum([(w / 100) * c[2] for w, c in zip(weights, colors)])
    
    if as_tuple:
        return (r, g, b)
    else:
        return f"rgb ({r}, {g}, {b})"


mix_color([20,30,50])

'rgb (129.0, 116.1, 113.9)'

In [91]:
def build_ce_ternary(data: pd.DataFrame, labels=dict(a='A', b='B', c='C'), mix_colors=['#1b9e77', '#d95f02', '#7570b3']) -> go.Figure:
    """
    Build the ternary plot for the given dataframe.
    """
    fig = make_subplots(rows=2, cols=2, specs=[[{'rowspan': 2}, {}], [None, {'type': 'ternary'}]])
    colors = [mix_color(row.iloc[:3].values, colors=mix_colors) for _, row in data.iterrows()]
    
    fig.add_trace(go.Scatterternary(
        mode='markers',
        a=[row.iloc[0] for _, row in data.iterrows()],
        b=[row.iloc[1] for _, row in data.iterrows()],
        c=[row.iloc[2] for _, row in data.iterrows()],
        marker=dict(
            size=14,
            color=colors
        ),
        hovertemplate="<br>" + labels['a'] +  ": %{a}%<br>" + labels['b'] + ": %{b}%<br>" + labels['c'] + ": %{c}%<extra></extra>",
        showlegend=False
    ), row=2, col=2)

    fig.add_trace(go.Scatter(
        x=data.meanCE,
        y=data.varCE,
        mode='markers',
        showlegend=False,
        marker=dict(
            size=14,
            color=colors
        ),
        text=[f"{labels['a']}: {row.iloc[0]}%<br>{labels['b']}: {row.iloc[1]}%<br>{labels['c']}: {row.iloc[2]}%" for _, row in data.
        iterrows()],
        hovertemplate="%{text}<extra></extra>"
    ))

    fig.update_layout(
        template='none',
        ternary=dict(
            sum=100,
            aaxis=dict(title=labels['a']), 
            baxis=dict(title=labels['b']), 
            caxis=dict(title=labels['c'])
        ),
        xaxis=dict(title="Variance"),
        yaxis=dict(title="Mean")
    )

    fig.add_trace(go.Bar(
        y=[0.1 for _ in range(len(colors))],
        x=[_ for _ in range(len(colors))],
        marker_color=colors,
        text=[f"{labels['a']}: {row.iloc[0]}%<br>{labels['b']}: {row.iloc[1]}%<br>{labels['c']}: {row.iloc[2]}%" for _, row in data.iterrows()],
        hovertemplate="%{text}<extra></extra>",
        showlegend=False
    ), row=1, col=2)

    fig.update_layout(
        yaxis2=dict(domain=[0.65,0.70], zeroline=False, visible=False),
        xaxis2=dict(domain=[0.65, 0.9], zeroline=False, visible=False),
        margin=dict(t=4, r=10),

    )
    
    return fig

build_ce_ternary(CEs, dict(a='Weizen', b='Mais', c='Grünland'))

In [3]:
fig = make_subplots(
    rows=1, cols=2,
    column_widths=[0.6, 0.4],
    specs=[[{"type": "scatter"}, {"type": "pie"}]])

fig.add_trace( go.Scatter(x=CEs.varCE, y=CEs.meanCE, mode='markers', name='Realisations'), row=1, col=1) 
fig.add_trace( go.Pie(values=CEs.loc['hay.70_wheat.10_maize.20',['wheat_perc','maize_perc','hay_perc']], name='BAU'),row=1, col=2) 

fig.update_layout(template='none', xaxis_title='Variance', yaxis_title='Mean')
fig.show()

In [ ]:
#does not work as expected

fig = make_subplots(rows=1, cols=1, specs=[[{"type": "pie"}]])
#fig.add_trace( go.Scatter(x=CEs.varCE, y=CEs.meanCE, mode='markers' ), row=1, col=1) 
for i in CEs.index:
    fig.add_trace( go.Pie(values=CEs.loc[i,['wheat_perc','maize_perc','hay_perc']], name=i, textinfo='none',
                          hoverinfo='none', domain={'x': [(CEs.loc[i,['varCE']].values[0]-5.3)/4.8-0.1, (CEs.loc[i,['varCE']].values[0]-5.3)/4.8+0.1], 
                                                    'y': [(CEs.loc[i,['meanCE']].values[0]-28.)/7.-0.1, (CEs.loc[i,['meanCE']].values[0]-28.)/7.+0.1]}),row=1, col=1) 
fig.update_layout(title='Title', template='none', xaxis={'range':(5.3,10.1)}, yaxis={'range':(28.,35.)} )
